In [1]:
from IPython.core.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

## 0) Some essential background


### Converting from base 10 to other bases

We only need to be able to convert from base from base 10 to base 2.

*(Incidentally, the ``converter`` below will also convert to other bases &#8804; 10)*


In [2]:
def eulidean_division(x, y):
    return x // y, x % y

def verif(a, base):
    return sum([int(val) * base ** int(pos) for pos, val in enumerate(str(a)[::-1])])

# positive integers and base <= 10 only
# (we don't include special symbols for higher bases)
def converter(a, base):
    q, r = eulidean_division(a, base)
    xs = [str(r), ]
    while q > 0:
        q, r = eulidean_division(q, base)
        xs.append(str(r))
    result = int(''.join(xs[::-1]))
    assert verif(result, base) == a
    return result

In [3]:
print(converter(194759, 2))

101111100011000111


### Euclid's extended algorithm

Instead of implementing Euclid's extended algorithm ourselves, we decide to rely on the `egcd` library for functionality and focus on a few interesting outcomes of the theorem:
- Bézout's identity
- Modular inverses

In [4]:
from egcd import egcd 

def euclidAlg(fst, sec):
    # egcd would not be difficult to implement but we focus on the resulting properties:
    # a) Bézout's identity
    # b) Modular inverses
    
    gcd, fst_inv, sec_inv = egcd(fst, sec)
        
    # a) Bézout's identity
    assert gcd == (fst * fst_inv) + (sec * sec_inv)
    
    # b) Modular inverses
    assert gcd == fst * fst_inv % sec
    assert gcd == sec * sec_inv % fst
    
    return {'gcd' : gcd, 'fst_inv' : fst_inv % sec, 'sec_inv' : sec_inv % fst}

In case that the input numbers $(a, b)$ are coprimes (a very common situation in this notebook), their greatest common divisor is $\text{gcd}(a, b) = 1$ and we have the following identities:

\begin{equation}
a \cdot a^{-1}_{\%b} + b \cdot b^{-1}_{\%a} = 1 \quad
\text{where} \quad  
\begin{pmatrix}
a \cdot a^{-1}_{\%b} \, \equiv 1 \,\,\text{mod}\,\,b \\
b \cdot b^{-1}_{\%a} \, \equiv 1 \,\,\text{mod}\,\,a
\end{pmatrix}
\end{equation}

Note that $\text{gcd}(a,b)$ and the modular inverses $a^{-1}_{\%b}$ and $b^{-1}_{\%a}$ are all computed efficiently and simultaneously with `egcd`. 

### Fast modular exponentiation

We are following a classic *square and multiply* method.  

(Numbers never exceed the modulus and the complexity only grows linearly with the number of bits of the modulus.)

In [5]:
def multiplyList(xs, m) :
    result = 1
    for x in xs:
        result = result * x % m
    return result % m

def FasterModularExponentiation(b, e, m):
    def helper(kmax):
        d = {}
        s = b % m
        d[0] = s
        for _ in range(kmax):
            s = (s * s) % m
            d[_ + 1] = s
        return d

    e2 = converter(e, 2)
    necessaryBits = [_[0] for _ in enumerate(reversed(str(e2))) if _[1] == '1']
    d = helper(max(necessaryBits))
    allVals = [d[_] for _ in necessaryBits]

    return multiplyList(allVals, m)

In [6]:
FasterModularExponentiation(14, 9, 138)

44

Quick check with small integers

In [7]:
14 ** 9 % 138

44

# 1) Real-world SSH keys

We start by using **ssh-keygen** to generate a pair of public / private keys into directory _ssh_key_example_.  By default the private key **ssh_key** (default name when generating the key) is in PEM (Privacy-Enhanced Mail) format but the public key is not.  Given the private key, one can generate a PEM public key by running:

```openssl rsa -in ssh_key -pubout -out ssh_key.pub.pem```

This way the contents of both keys can be parsed with openssl.

## 1.1) Public Key

The public key is composed of 2 integers:

---
- the modulus denoted as $n$,
- and the public exponent $e$
---

Both are represented below in hexadecimal representation.

<img src="figs/publicKey.jpg">

In [8]:
pem_public_n = int('C4BBED112C039C1A071711E9A8DEDB95C3287169C9719D5561F85B82CDF0543FDD19EE794D91AAB4AD570F020C38\
A64DD40E0AAF52733AF4EB7E6B45E216A1923CC247EBF7CB6A616D18AE22AC5CB499E49B2C797A949E91FACC33EBF35192695610D5EF3546\
6D0AA36B7EED7C83953EE5B21D3712CC7394F57D8CF542207494339AE28C61D4F47007613563BB3046CF2787C86C08BEDB1B51C12F11EF55\
797DDEEFC6BD47CDC9EC110D69DC725F28006EA23B2984D66328615400D2115360CA3765394DE4BAAD5D8A42E64D1128A3FC948D81ABAD2A\
DC8DBD0383F1EDF5D5A423D3F847029484D8D570B4F8612EA2C237199693DBC3CB9C0C441007C884F34B', 16)

print('public modulus =', pem_public_n)

pem_public_e = int('010001', 16)

print('\npublic exponent =', pem_public_e)

public modulus = 24835377559135552038795062294607604159367680484145008279339224671520046965058529987929829903212777617470542604231099658464774726678493177255047233626637210508539440468877263390150782180405854795055249603047135471961292988773672924415714269315236257502478131495369758902277205183907767167824428219178242912233154757059128223509520497483406485754976556841768412218361011069744928326674424583481946213409954408380026753728324536242688309296947434127561960371528514451037596617839124259123219608145478120141266515401573876505210816544126435618294622284092184314366686115194625300871169747984813911234207139252276742648651

public exponent = 65537


By convention, the public exponent is fixed to a 65537.  This value is rather small when compared to the modulus and the notebook will have a section to below to discuss the order of magnitudes of the different numbers.  We will check later (when we have access to the private key) that this value is indeed coprime to $\phi(n)$ as should be for RSA to work)

## 1.2) Private key

Obviously, the private key contains more information which is composed of the following 5 integers:

---
*(repeated from public key)*
- the modulus $n$
- the public exponent $e$
---
- the private exponent $d$ used for decrytion
- the first random prime $p$
- the second random prime $q$
---
*(to be used for optimized decryption via Chinese remainder theorem)*
- the first optizmized exponent $d_p$
- the second optizmized exponent $d_q$
- modular inverse $q^{-1}_{\%p}$
---

<img src="figs/privateKey.jpg">

We start by verifying that $n$ are $e$ are indeed identical between the public and private keys:

In [9]:
pem_private_n = int('C4BBED112C039C1A071711E9A8DEDB95C3287169C9719D5561F85B82CDF0543FDD19EE794D91AAB4AD570F020C38\
A64DD40E0AAF52733AF4EB7E6B45E216A1923CC247EBF7CB6A616D18AE22AC5CB499E49B2C797A949E91FACC33EBF35192695610D5EF35466\
D0AA36B7EED7C83953EE5B21D3712CC7394F57D8CF542207494339AE28C61D4F47007613563BB3046CF2787C86C08BEDB1B51C12F11EF5579\
7DDEEFC6BD47CDC9EC110D69DC725F28006EA23B2984D66328615400D2115360CA3765394DE4BAAD5D8A42E64D1128A3FC948D81ABAD2ADC8\
DBD0383F1EDF5D5A423D3F847029484D8D570B4F8612EA2C237199693DBC3CB9C0C441007C884F34B', 16) 

pem_private_e = int('010001', 16)

In [10]:
assert pem_private_n == pem_public_n
assert pem_private_e == pem_public_e

The following three consist of the private exponent $d$ and the two underlying primes $p, q$ that divide the factorize the (public) modulus

In [11]:
pem_private_d = int('43C09C922F6046E047D4D01C7245DDC5A3E247BCB0DD69DBBFD3B26047E01E83A7F788434A4D82469AF3C27D680C\
1269F73FA6BA5E60C4CB1856FF469FE83F5887883910B0D3E31D0E5F53892966FBE38BF14CCDC14371A0C08896D10988EC2D4EB0999CB9F23\
91752D470700553A39077B6718D47F21554157F598E1569EBFB8B51D7286D600E1032F0E47F6E96590DBB9B21B3F880EA9DCE066B312DB0B7\
FBB26EBC8C566564E529D051CC6B1D137BA6CC087ADC12A26BF7336C06FED53F2595DE644C98894829216BD1B4001C8FE69CD5B384339BB8E\
C6A478398A9973716A382065F4CAA64CE89C30AE99E88503A59D41807A55A8C9800C043DFFFE89491', 16)

pem_private_p = int('EB5EEF3FB761F094956486B80AEEB43310CA31B59818B2CB5897B7C4BF7CDDF1DAA5266B7A970CC9895BD9FDA8D4\
2DA5D8F1D12CBD67DACD2DF039875811B589A207CA3566CDDC3581CC6BCDF7F39E9FBEB54DBC85AD9E763800556C81229D42360EA45AB9830\
5C3933CE32AC629DC93895966C2EED55575D7E695F760FC87C7', 16)

pem_private_q = int('D5FA1729510AD44E7DF99A8001F27F3690E97DF36A381994E7D9CE24E93A9C44A525522E271E10CA6FAD1323CEAC\
E86CA9567FBB70A50FCD2CA2F7042FB12A75D0473366792F6F55856C70618AC3018F9891A03BBFA9333397066D557A6FFD0A5008DCC436441\
704373A46F0DD42722C1D5B14D8BC7393DC3C7BBB41F93B605D', 16)

print('p =', pem_private_p)
print('\nq =', pem_private_q)
print('\nd =', pem_private_d)

p = 165283023393017258802844128042087187812519603574924144141096672182734181606686932634447869790251125699666869503105901189823854879217946475557773558838848560824845909085099897384473466588627428165736863357777495899717471740241145780518183395624614152064709237404036084856914835566384405188943296394177327957959

q = 150259700296508350526263337597940315265703885659861438663516643854644969839622160478192820947916953384756820325711611538308862848617615705228470107601719502305112976164205360469405751670065426938606654576750244537701551043712022710518528333562390395012907956412099441425756277336490106954095150955492592541789

d = 855294675541586294025671843369842418598545170708382801874797901698593863010774099863552284839880358921388141931269236143781322888038193708357746102130402430959206511409676724164522565591589701579866309932975033343250961680610644222443308449341413753804616366099295754191367503854003547580446686462384519476166411443231840657349771385173244064795986059427016728206216

We can check that the modulus $n = pq$ is indeed the product of the private random primes.  In addition, we also check that the public exponent $e$ is coprime to Euler's totient function $\varphi(n)$. Since $p$ and $q$ are primes, we have $$\varphi(n) = \varphi(p)\cdot\varphi(q) = (p-1)\cdot(q-1)$$.

In [12]:
assert pem_public_n == pem_private_p * pem_private_q
assert 1 == euclidAlg(pem_private_e, (pem_private_p - 1) * (pem_private_q - 1))['gcd']

Therefore we can verify that the private exponent is indeed the inverse of the public exponent modulo $\varphi(n)$

\begin{equation}
e \cdot d \equiv 1 \,\, \text{mod} \,\, \varphi(n)
\end{equation}

In [13]:
assert 1 == pem_public_e * pem_private_d % ((pem_private_p - 1) * (pem_private_q - 1))

The other numbers contained in the private key are not strictly necessary for decryption.  They will come in handy when using the Chinese remainder theorem to optimize the decryption part (discussed further below in the notebook).

In [14]:
pem_crt_exponent_p = int('090DD0D439A0A9D97D59AD98FCCAABE1DEFE78250D21BE16F66477AC38D57BD026E1FD755AF4DE880E219B6\
4178F79E60C4F5905888FAB2A035F5F47268B1FD99883063341AE1F8F6B5BCAB5D498E97C29A0DDC9A7B6C306B726C4227561387D6888EDA7\
93C5477E7B7677229916D9264FBD41A7B3FA3EDC569EC71C09FE0CD7', 16)

pem_crt_exponent_q = int('A78500150CA999C549C2DC3E5347F26859D333141A98890E96E5620A6BBDC311CAE90909B0FE4CCEEFD7642\
0A8719E15540DB03AA2D9D7211E4D076E73981451E0199E181FD00F2DB923486855268E56496FD92E6DC29D9F4A2171FF75B7AF371FA9908C\
DC9F3A15C6D70D2B0D4E33EC6D9F1D170192CAB3A5768908018386D1', 16)

pem_crt_qInv = int('D0D4AC811099F358DF4D00ACAE97CF6FCD4C8CE92DD384929E07ACA8E97836754BF69BE3BA84B536989332C2BDEFF\
B2026D651F6586678F644BB93BD0B5148F52DFFAC8EEA6034B11AB989F7CF2CAA32C35221030DB13E2A4210F2C8FE30C46B9B68C1FEF4D175\
BFF44A00E9AFED8C8A4ECD9F0F2A6C56F79FEBE9BDA722C746', 16)

# 2) How big are these numbers?

Before going on to RSA, it is interesting to pause and try to get a feel for the scale of the numbers that we are discussing here.  

It will be useful to have a function to approxiamte numbers as powers of 10. 

In [15]:
from math import log10, log, floor

def approx10(x): # take a power an exponent in base 2 and approximate in base 10
    b10exp = floor(log10(x))
    prefactor = round(x / 10 ** b10exp, 3)
    return '%s x 10^{%s}' % (prefactor, b10exp)

Let us start by looking at the two underlying random primes $p$ and $q$.

In [16]:
print('p = %d\n' % pem_private_p)
print('p ~ %s\n' % approx10(pem_private_p))

bin_p = bin(pem_private_p)[2:]

print('p = %s\n' % bin_p)
print('number of bits in p = %d' % len(str(bin_p)))

p = 165283023393017258802844128042087187812519603574924144141096672182734181606686932634447869790251125699666869503105901189823854879217946475557773558838848560824845909085099897384473466588627428165736863357777495899717471740241145780518183395624614152064709237404036084856914835566384405188943296394177327957959

p ~ 1.653 x 10^{308}

p = 11101011010111101110111100111111101101110110000111110000100101001001010101100100100001101011100000001010111011101011010000110011000100001100101000110001101101011001100000011000101100101100101101011000100101111011011111000100101111110111110011011101111100011101101010100101001001100110101101111010100101110000110011001001100010010101101111011001111111011010100011010100001011011010010111011000111100011101000100101100101111010110011111011010110011010010110111110000001110011000011101011000000100011011010110001001101000100000011111001010001101010110011011001101110111000011010110000001110011000110101111001101111101111111001110011110100111111011111010110101010

In [17]:
print('q = %d\n' % pem_private_q)
print('q ~ %s\n' % approx10(pem_private_q))

bin_q = bin(pem_private_q)[2:]

print('q = %s\n' % bin_q)
print('number of bits in q = %d' % len(str(bin_q)))

q = 150259700296508350526263337597940315265703885659861438663516643854644969839622160478192820947916953384756820325711611538308862848617615705228470107601719502305112976164205360469405751670065426938606654576750244537701551043712022710518528333562390395012907956412099441425756277336490106954095150955492592541789

q ~ 1.503 x 10^{308}

q = 11010101111110100001011100101001010100010000101011010100010011100111110111111001100110101000000000000001111100100111111100110110100100001110100101111101111100110110101000111000000110011001010011100111110110011100111000100100111010010011101010011100010001001010010100100101010100100010111000100111000111100001000011001010011011111010110100010011001000111100111010101100111010000110110010101001010101100111111110111011011100001010010100001111110011010010110010100010111101110000010000101111101100010010101001110101110100000100011100110011011001100111100100101111011011110101010110000101011011000111000001100001100010101100001100000001100011111001100010010001101

The primes have 1024 bits and therefore are on the order to $10^{308}$.  As the product of $p$ and $q$, the modulus $n = pq$ is a 2048-bit number on the order of $10^{616}$.

In [18]:
print('n = %d\n' % pem_public_n)
print('n ~ %s\n' % approx10(pem_public_n))

bin_n = bin(pem_public_n)[2:]

print('n = %s\n' % bin_n)
print('number of bits in n = %d' % len(str(bin_n)))

n = 24835377559135552038795062294607604159367680484145008279339224671520046965058529987929829903212777617470542604231099658464774726678493177255047233626637210508539440468877263390150782180405854795055249603047135471961292988773672924415714269315236257502478131495369758902277205183907767167824428219178242912233154757059128223509520497483406485754976556841768412218361011069744928326674424583481946213409954408380026753728324536242688309296947434127561960371528514451037596617839124259123219608145478120141266515401573876505210816544126435618294622284092184314366686115194625300871169747984813911234207139252276742648651

n ~ 2.484 x 10^{616}

n = 110001001011101111101101000100010010110000000011100111000001101000000111000101110001000111101001101010001101111011011011100101011100001100101000011100010110100111001001011100011001110101010101011000011111100001011011100000101100110111110000010101000011111111011101000110011110111001111001010011011001000110101010101101001010110101010111000011110000001

### How many 1024 bit numbers are there? 

The smallest and largest representable numbers with $N$ bits and no fewer are given by:

\begin{equation}
1 \big( \underbrace{ 0 \cdots 0}_{N-1 \text{ bits} }  \big)  \leq \,\, \text{all possible numbers} \,\, \leq \,\, 1 \big( \underbrace{ 1 \cdots 1}_{N-1 \text{ bits} }  \big)  
\end{equation}

Obviously, the most significant bit must always set to 1 otherwise the number could be represented using less than $N$ bits.  Therefore, the number of representable numbers that have exactly $N$ bits is determined by the number variations using the remaining $N-1$ bits, i.e. there are $2^{N-1}$ numbers that have exactly $N$ bits.

In [19]:
approx10(2 ** 1023)

'8.988 x 10^{307}'

This means that there are about $9 \cdot 10^{307}$ numbers that have 1024 and no fewer bits.

All numbers with 1024 bits are bounded from below by $2^{1023} \approx 0.9\cdot 10^{308}$ (smallest possible number) and from above by $2^{1024} - 1 \approx 2^{1024} \approx 1.8 \cdot 10^{308}$ (largest possible number).  Obviously, the maximum number is always (regardless of $N$) just twice the smallest possible number. The fact that there are so many numbers in this interval is simply because the bounds of the interval are so big to begin with.

In [20]:
assert 2 ** 1023 <= pem_private_p < 2 ** 1024
assert 2 ** 1023 <= pem_private_q < 2 ** 1024
assert 2 ** 2047 <= pem_public_n < 2 ** 2048

### How many of these are primes?

The prime number theorem states that the number of primes $\pi(x)$ less than or equal to $x$ converges to:

\begin{equation} 
\lim_{x \rightarrow \infty} \pi(x) = \dfrac{x}{\log x}
\end{equation}

Therefore the number of primes with $b=1024$ bits (and no fewer) can be estimated by:

\begin{equation}
\pi \left( 2^{1024} \right) - \pi \left( 2^{1023} \right) \approx \dfrac{2^{1023}}{1024\log2} \approx 1.3 \cdot 10^{305}
\end{equation}

Even though, only about 0.14% of all 1024-bit numbers are primes, this still leaves more than $\sim 10^{305}$ primes. Clearly, it is not possible to store / enumerate these numbers.

In [21]:
print(approx10(2 ** 1023))
print(approx10(1024 * log(2)))
print(round(9 / 7.1, 1))
print(307 - 2)
print(round(100 * (1.3 * 10 ** 305) / (9 * 10 ** 307), 2))

8.988 x 10^{307}
7.098 x 10^{2}
1.3
305
0.14


# 3) RSA encryption / decryption

## Pycrypto

Instead of using the more modern *Pycryptodome* library, we refer to the deprecated [*Pycrypto*](https://pypi.org/project/pycrypto/).  

The reason is that Pycrypto exposes the raw decrypt / encrypt methods.  This is considered very bad practice as it leads to easy attacks on RSA.  Instead, one should use the Crypto.Cipher.PKCS1_OAEP (the only option in pycryptodome) which securely takes care of the padding.  Details about the OAEP padding can be found in the accompanying PDF document.

However, since our intention is to understand the bare bone mechanics of textboook RSA, it is very useful to be able to use these raw decrypt / encrypt methods.

In [22]:
from Crypto.PublicKey import RSA
from Crypto.Util.number import long_to_bytes, bytes_to_long

First of all, load the ssh keys using Pycrypto

In [23]:
ssh_public_key  = RSA.importKey(open('ssh_key_example/ssh_key.pub').read())
ssh_private_key = RSA.importKey(open('ssh_key_example/ssh_key').read())

Verify that we are indeed reading the same keys as what we saw previously from the PEM format

In [24]:
assert ssh_public_key.n == ssh_private_key.n
assert ssh_public_key.e == ssh_private_key.e

assert ssh_private_key.n == pem_private_n
assert ssh_private_key.d == pem_private_d
assert ssh_private_key.p == pem_private_p
assert ssh_private_key.q == pem_private_q

Once, again show that basic identities are holding:

In [25]:
assert ssh_public_key.n == ssh_private_key.p * ssh_private_key.q
assert 1 == ssh_public_key.e * ssh_private_key.d % ((ssh_private_key.p - 1) * (ssh_private_key.q - 1))

## 3.1) Plaintext message

### How to encode text-based messages into integers?

Let's define our plaintext message as a string and convert it to a number.  This can be done by converting each character of the message into a 8-bit ASCII binary code and appending all the codes one after the other. The final sequence of bits can then be converted into an integer as shown in the small example below:

In [26]:
import pandas as pd
from itertools import accumulate

short_example_message = "Red Queen's race"

df = pd.DataFrame({'Character' : [c for c in short_example_message]})
df['ASCII'] = df['Character'].apply(lambda c: bin(ord(c))[2:].zfill(8))
df['Accumulated message'] = list(accumulate(df['ASCII'], lambda x,y: str(x)+str(y)))

extraInfoGap = pd.Series({'Character': '', 'ASCII': '', 'Accumulated message': ''})

extraInfo = pd.Series({'Character': u'\u27A5', 
                       'ASCII': r' $\texttt{Red Queen`s Race}$', 
                       'Accumulated message': 'converting from base \u2777 to base \u277F yields  ' + 
                       str(verif(df.loc[15]['Accumulated message'],2)) +
                       u' \u2248 1.095 \u00D7 10\u00B3\u2078'})

df.loc[16] = extraInfoGap
df.loc[17] = extraInfo

dfStyler = df.style.set_properties(subset=["Accumulated message"], **{'text-align': 'left'}).set_properties(subset=["Character", "ASCII"], **{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

,Character,ASCII,Accumulated message
0,R,01010010,01010010
1,e,01100101,0101001001100101
2,d,01100100,010100100110010101100100
3,,00100000,01010010011001010110010000100000
4,Q,01010001,0101001001100101011001000010000001010001
5,u,01110101,010100100110010101100100001000000101000101110101
6,e,01100101,01010010011001010110010000100000010100010111010101100101
7,e,01100101,0101001001100101011001000010000001010001011101010110010101100101
8,n,01101110,010100100110010101100100001000000101000101110101011001010110010101101110
9,',00100111,01010010011001010110010000100000010100010111010101100101011001010110111000100111


Taking the final binary string and converting it to an integer yields:

In [27]:
short_example_message_int = verif(df.iloc[15]['Accumulated message'], 2)

print(short_example_message_int)
print(approx10(short_example_message_int))

109523148438546893738592179701212603237
1.095 x 10^{38}


which we can verify using some built-in (and far more efficient) **bytes_to_long** Pycrypto methods

In [28]:
bytes_to_long(short_example_message.encode('utf-8'))

109523148438546893738592179701212603237

### More realistic (longer) plaintext message

Now that we understand how to transform a text message into an integer, let's consider a longer more realistic message:

In [29]:
msg = b'Now, here, you see, it takes all the running you can do, to keep in the same place. \
If you want to get somewhere else, you must run at least twice as fast as that!'

msgInt = bytes_to_long(msg)
print(msgInt)

107000686390655777318175324491215136984667835563811914458266416155904255907525235966061730904317431448927902977328707943659320485148403350342199675264281892239196035922708980386263850710869034556829889362966321823614168001147021967461629724821385450564072293287067084372107029031933850136051297187681789299078932638400396786359008268912113436515860008110465019951840714333167627203495615099937


We should make sure that our message is not bigger than the modulus $n$ otherwise it should be broken up into smaller chunks

In [30]:
print('message = ', approx10(msgInt))
assert msgInt < ssh_public_key.n

message =  1.07 x 10^{392}


Encoding each character as a 1 byte, our message has 163 bytes so it does not need to be cut into smaller chunks.  The maximum message size possible with RSA-2048 is bounded by the public modulus 2048 bits = 256 bytes.

In [31]:
len(msg)

163

*(For context, we can also see that the message is larger than any of the individual primes.  Not that this is actually important...  In case, the message would be smaller, the recombining step during decryption using CRT collapses to a trivial special case.*)

In [32]:
print(msgInt > ssh_private_key.p)
print(msgInt > ssh_private_key.q)

True
True


## 3.2) Encryption

The cipher is determined by the remainder of the division of the plaintext $m$ raised to the public exponent $e$ by the public modulus $n$

\begin{equation}
c = m^e \,\,\text{mod}\,\, n
\end{equation}

In the cells below, we show that our simple implementation of modular exponentiation is able to reproduce the encrypted message generated from the Pycrypto library.

### with Pycrypto

In [33]:
cipher = ssh_public_key.encrypt(msgInt, 'x')[0]
print('c = ', cipher)

print('\nc =', approx10(cipher))
print('\nhex representation of cipher = ', hex(cipher))

c =  12900815517135424771274705545216893949162249854475773699373379383283322111815909664428418213578492577956719649336028503213693001899110981344082059534316812974205798626750761899328504466635981656142944027003448793428470452978612397457872229686949112855238535167400487771395995839716017888521696728217479279241923242560099823444406456022112114125096669117362497106689593606180465467176718841622909774903307841867976290293998434632844564867045116909170721515885995688719473914190485526954084354476385592238310399295983736841264408856400224102841175920822604234988947904291035343015209859670304020885301115658025156703529

c = 1.29 x 10^{616}

hex representation of cipher =  0x6631b36bf8c182d99d4815d789e8c90a8208f09af7d5992245be7c1a572256372fed47bf22acba76139798294ea06901e568d3350c9b1c3701bf332898f406b21c934e24fe289bcf6eb2f07c9a937d60841dd7bc19d33e643420ebadcdafb961fa474247331c2367aa7d878e78f87d2f10f81a14eda3f3493a8fe3fd53c97480c980424068f499f58a515efce8bff0980f9512f585a4a7393753432d1b79344cd

### with custom modular exponentiation

We use the square and multiply modular exponentation function implemented earlier in the notebook

In [34]:
my_cipher = FasterModularExponentiation(msgInt, ssh_public_key.e, ssh_public_key.n)

And we can verify that this cipher is indeed identical to the one produced by the library

In [35]:
assert my_cipher == cipher

### Some side-note magic...

The cipher is just the remainder of $m^e$ after division with $n$.  Because of this, the cipher is upper bounded by $n \sim 10^{616}$.    As big as this number may be, it is completely negigible compared to the full value of $m^e$.  Considering a typical message $m \sim 10^{392}$ such as the one above, the exponentiation to $e$ leads to a number on the order to $m^e \sim 10^{392 \, \cdot \, e} \sim 10^{25,690,504}$ which is more than 40,000 orders of magnitude bigger than the cipher.


\begin{equation}
m^e \,\,\text{mod}\,\,n \,\,\sim \,\, 10^{616} \underbrace{\,\,\,\, \ll \,\,\,}_{\text{understatement} \,\,\,\,} m^e \,\, \sim \,\, 10^{25,690,504} \underbrace{\,\,\,\, \approx \,\,\,\,}_{\text{in some sense...}} 10^{616} \cdot 10^{41,705}
\end{equation}

Even though the cipher throws away almost all the information from $m^e$ by keeping only the remainder after division with $n$, this is still enough for the decryption to work and recover the original message.

In [36]:
print('c / n = %.2f' % (cipher / ssh_private_key.n))
print('c ~ %s' % approx10(cipher))

print('m ~ %s' % approx10(msgInt))
print('392 x e = %d' % (392 * ssh_public_key.e))

c / n = 0.52
c ~ 1.29 x 10^{616}
m ~ 1.07 x 10^{392}
392 x e = 25690504



## 3.3) Decryption

Similar to the encryption, we will show first how to decode the cipher using the Pycrypto library and then verify that our custom modular exponentiation function reproduces the same result.  The message can be recovered by evaluating the remainder of the division of the cipher $c$ raise to the private exponent $d$ with the public modulus $n$:

\begin{equation}
c^d \,\,\text{mod}\,\, n \equiv m \,\,\text{mod}\,\,n
\end{equation}

The proof of why this is true is given in the accompanying PDF document.  Note that the private exponent $d$ is very large (on the order to the modulus $n$) since it is defined as the modular inverse of the public exponent $e$ which is itself quite small:

\begin{equation}
e \cdot d \equiv 1 \,\,\text{mod}\,\, \phi(n)
\end{equation}


#### The private exponent $d$

One of the strenghts of RSA is that it is not feasible to evaluate Euler's totient function $\phi(n)$ without having previous knowledge of $p$ and $q$.  In other words, calculating $\phi(n)$ is as hard as factorizing $n$ into its prime factors.  However, if we do know the values of $p, q$ it is very easy to calculate $\phi(n)$ as

\begin{align}
\phi(n) &= \phi(p) \cdot \phi(q) \\
&= (p-1) \cdot (q-1)
\end{align}

In [37]:
phiN = (ssh_private_key.p - 1) * (ssh_private_key.q - 1)

First of all, we shoud verify that $e$ and $\phi(n)$ are co-primes (actually it was already checked earlier but we can do it again...).  This is a necessary condition for the modular inverse of $e$ to exist (and be unique).

In [38]:
egcd_e_phiN = euclidAlg(ssh_public_key.e, phiN)

assert egcd_e_phiN['gcd'] == 1

In this case, $d$ is also calculated using Euclid's extended theorem.

In [39]:
my_private_d = egcd_e_phiN['fst_inv']

assert my_private_d == ssh_private_key.d

print('e = %d' % ssh_public_key.e)
print('\nd = ', my_private_d)
print('\nd ~ %s' % (approx10(ssh_private_key.d)))
print('\nd / n ~ %s' % approx10(ssh_private_key.d / ssh_public_key.n))

e = 65537

d =  8552946755415862940256718433698424185985451707083828018747979016985938630107740998635522848398803589213881419312692361437813228880381937083577461021304024309592065114096767241645225655915897015798663099329750333432509616806106442224433084493414137538046163660992957541913675038540035475804466864623845194761664114432318406573497713851732440647959860594270167282062161791067326622898493471132874950740836740348227125932097494861277136605503073833692844134364327783924735395647530629394830593401357148357162397620141771686986904880710295543924974308012419051035511017524642785181842534656749198734877662637011702944913

d ~ 8.553 x 10^{615}

d / n ~ 3.444 x 10^{-1}


With $e = 65537$, we have a private exponent $d \approx 8.5 \cdot 10^{615} \approx 0.35 \, n$. Because this decryption exponent is so big, it is more efficient to optimize the decryption using the Chinese remainder theorem than to perform a straightforward exponentiation directly on $c$.  

We will show in a section below how to do this optimization, but for now in the sake of simplicity, let us go ahead and calculate $c^d \,\,\text{mod}\,\,n$ directly.

### with Pycrypto

In [40]:
decrypt_msg = ssh_private_key.decrypt(cipher)
decrypt_msg = long_to_bytes(decrypt_msg)
print(decrypt_msg)

b'Now, here, you see, it takes all the running you can do, to keep in the same place. If you want to get somewhere else, you must run at least twice as fast as that!'


### with custom modular exponentiation

In [41]:
my_decrypt_msg = FasterModularExponentiation(my_cipher, ssh_private_key.d, ssh_public_key.n)

my_decrypt_msg = long_to_bytes(my_decrypt_msg)
print(my_decrypt_msg)

b'Now, here, you see, it takes all the running you can do, to keep in the same place. If you want to get somewhere else, you must run at least twice as fast as that!'


Verify that the initial message is indeed recovered by all methods

In [42]:
assert msg == my_decrypt_msg == decrypt_msg

### Sanity check

As a sanity check, let us change the private exponent $d$ into a fake $d_\text{fake} = d - 1$ almost identical to the real one.  Notice that we are talking about a difference of 1 to numbers on the order of $10^{615}$ and yet, as expected, the decryption completely fails to recover the original message.

In [43]:
long_to_bytes(FasterModularExponentiation(my_cipher, ssh_private_key.d - 1, ssh_public_key.n))

b']q3\xd6\x12b\xb7\xb6\x83\xe0\x84s\xf0o_\t}+\xb24\xc7\xd2\xa1\x177!:\x84\xe5\xa1,$\xa90\x7f\xcb,\xc3\xbe`\xe5\xac3CWkf\x96<:n\xd5:\x013?m\x07\x10G\xab@\x10\xc4`U\xfa\xfa\x03T\xef\x92\xad\xec\xde\xa2M\x8b\xf2\xc9J\xd1F\xf2\xe1Z\xf6\x00\x82"&J34\x0b\xc0\xd2\x1a\x16|@*\xfc\xac\x82\x03k\xa0\x11\xde\x12v\x9c\xfa\xf64\xd2\x9a\xad\x0c|#6\xfbT?h\x98f\x18\xefk\xac*\xb5P\xcd\xaa\xdf#\xe8\xdd(\x9cB\xe7\x1e\xa4\xeco\xc5>7\xa1\x9e\x8a\xee\x97t"\xb4\xfb\xbc\x94H\xcf\xf4Siz\xaf\xe9\xae\xd6\xb6t\xdd\xbd\xcf\x17Z\x8b\x89\xeexd\x98\xae\xe6n\xbcV\x8d-\xf4\xb4\xa1\x91\xc6\xb2\x0c\xcf\x0b\x86EY]\xde\x89_@iUB1Z\\7\xe7\xb0\x1fA\xbc\x86\x82\x0e\x8a\xb6k\xbd/\x14\xfeLHqt\xfd\xc4\x80n\xc1\xac\xc9\x9f=\x0c\xa8\x93\xed\xf8\xe9\xf6\x12j\x99'

## 3.4) Optimizing the decryption using the Chinese remainder theorem

As we saw above, the decryption exponent $d$ is a very large number on the same order as the modulus itself $n$.  This puts significantly more computational pressure on the decryption part than on the encryption one.  This inefficiency can be mitigated by using an optimization technique based on the Chinese remainder theorem.

Since $p$ and $q$ are primes (and therefore co-primes between each other), we can separate numbers in $\mathbb{Z}_{pq}$ into a  unique system of congruences:

\begin{equation}
c \in \mathbb{Z}_{pq} \longrightarrow \big[ c\,\,\text{mod}\,\, p\,,\, c\,\,\text{mod}\,\,q \big] \in \mathbb{Z}_p \times \mathbb{Z}_q
\end{equation}

Therefore the exponentiation can be carried out independently on the two sides leading to:

\begin{equation}
m \equiv c^d \in \mathbb{Z}_{pq} \longrightarrow \big[ m_p = c_p^{d_p} \,,\, m_q = c_q^{d_q} \big] \in \mathbb{Z}_p \times \mathbb{Z}_q
\end{equation}

where the congruences $m_p$ and $m_q$ are determined by:

\begin{align} 
c_p = c \,\,\text{mod}\,\,p \quad \text{and} \quad  d_p &= d \,\,\text{mod}\,\, (p-1) \\
c_q = c \,\,\text{mod}\,\,q \quad \text{and} \quad  d_q &= q \,\,\text{mod}\,\, (q-1)
\end{align}

**(All details can be found in the accompanying PDF document)**

In [44]:
c_p = cipher % ssh_private_key.p
c_q = cipher % ssh_private_key.q

d_p = ssh_private_key.d % (ssh_private_key.p - 1)
d_q = ssh_private_key.d % (ssh_private_key.q - 1)

In [45]:
print('c_p = ', approx10(c_p))
print('c_p / c = ', approx10(c_p / cipher))

print('\nc_q = ', approx10(c_q))
print('c_q / c = ', approx10(c_q / cipher))

c_p =  4.939 x 10^{307}
c_p / c =  3.829 x 10^{-309}

c_q =  3.05 x 10^{307}
c_q / c =  2.364 x 10^{-309}


In [46]:
print('dp = %s' % approx10(d_p))
print('dp / d = %s\n' % approx10(d_p / ssh_private_key.d))

print('dq = %s' % approx10(d_q))
print('dq / d = %s' % approx10(d_q / ssh_private_key.d))

dp = 6.358 x 10^{306}
dp / d = 7.434 x 10^{-310}

dq = 1.176 x 10^{308}
dq / d = 1.375 x 10^{-308}


This shows that $d_p$ and $d_q$ are now bounded by the underlying primes instead of the modulus so that they are incredibly smaller (more than 300 orders of magnitude) than the original private exponent $d$.  

In addition, **we can verify that these new exponents $d_p$ and $d_q$ are indeed the ones contained in the PEM private key** we were looking at earlier:

In [47]:
assert d_p == pem_crt_exponent_p
assert d_q == pem_crt_exponent_q

In [48]:
m_p = FasterModularExponentiation(c_p, d_p, ssh_private_key.p)
m_q = FasterModularExponentiation(c_q, d_q, ssh_private_key.q)

The system of congruences:

\begin{align}
m &\equiv m_p \,\,\text{mod}\,\, p \\
m &\equiv m_q \,\,\text{mod}\,\, q
\end{align} 

is satisfied by a unique $m$ given by:

\begin{equation}
m \equiv \big( m_q p p^{-1}_{\%q} + m_p q q^{-1}_{\%p} \big) \,\,\text{mod}\,\,n
\quad \text{where the modular inverses} \quad
\begin{pmatrix}
p p^{-1}_{\%q} &\equiv 1 \,\,\text{mod}\,\, q \\
q q^{-1}_{\%p} &\equiv 1 \,\,\text{mod}\,\, p
\end{pmatrix}
\end{equation}

can be found efficiently using Euclid's extended algorithm.

In [49]:
egcd_crt = euclidAlg(ssh_private_key.p, ssh_private_key.q)

pInv = egcd_crt['fst_inv'] % ssh_private_key.q
qInv = egcd_crt['sec_inv'] % ssh_private_key.p

assert 1 == ssh_private_key.p * pInv % ssh_private_key.q
assert 1 == ssh_private_key.q * qInv % ssh_private_key.p

Using Bézout's identity $pp^{-1}_{\%q} + qq^{-1}_{\%p} = 1$, the decrypted message $m$ can be re-written as an expression:

\begin{equation}
m = m_q + q q^{-1}_{\%p} \big( m_p - m_q \big) \,\, \text{mod} \,\, n
\end{equation}

that involves only a **single modular inverse $q^{-1}_{\%p}$ which can be precomputed only once and kept directly inside of the PEM private key** for even faster decryption:

In [50]:
assert pem_crt_qInv == qInv % ssh_private_key.p

In [51]:
recombined_m = (m_q + ssh_private_key.q * qInv * (m_p - m_q)) % ssh_private_key.n

recombined_m = long_to_bytes(recombined_m)
print(recombined_m)

assert recombined_m == msg

b'Now, here, you see, it takes all the running you can do, to keep in the same place. If you want to get somewhere else, you must run at least twice as fast as that!'


# 4) Using RSA for digital signature

In the previous section we considered the (classic) scenario where one uses the public key to encrypt a message and the private key to decrypt.  Conceptually, the whole procedure can be summarized as:

\begin{equation}
\big(m^e\big)^d \equiv \big(m^e\big)^{d \sim 1/e} \equiv m \,\,\text{mod}\,\,n
\end{equation}

Clearly, the order of operation could be reversed:  encrypt the message by raising $m$ to the private exponent $d$ and send the cipher over for decryption by the public exponent

\begin{equation}
\big(m^d\big)^e \equiv \big(m^d\big)^{e \sim 1/d} \equiv m \,\,\text{mod}\,\,n
\end{equation}

This is useful for digital signatures.  Following the [example from Wikipedia](https://en.wikipedia.org/wiki/Digital_signature) where the sender (Alice) wants to send a message $\quad m= \texttt{Hello Bob}\quad $ to the recipient (Bob).  In order for Bob to be sure that the sender is really Alice, Bob can request that Alice signs the message.  Alice can do this by appending the encryption of her message to the message itself.  If Alice encrypts with the private key, anyone with the public key can decrypt the message.  If the decryption matches the message, it means that the sender used the correct private key which only Alice has access to.  Therefore, Bob can be sure that Alice is indeed the true sender of the message.   

<img src="figs/digitalSignature.png" style="width: 350px;">

In [52]:
message_to_sign = b'Hello Bob!'

message_to_sign_int = bytes_to_long(message_to_sign)
print(message_to_sign_int)

341881320659697045299745


The signature is obtained by encrypting the message **using the private key**

In [53]:
rsa_signature = FasterModularExponentiation(message_to_sign_int, ssh_private_key.d, ssh_private_key.n)

print(hex(rsa_signature))

0xc00a9d1b661c6456d1cb0d3aadbb71db0e43c91a717cd7a062c9f96e302485cd9df2810b01c04b68216abf5a5405951f55d24c0e82d975a37987b462b49d116d3197485e9e25c2583c11281a42776079d1cacb4883ef8e1179c030c1658fb1614f091f6c7b6d65da2f057048ff7a2c6460f79c4c5639d4d863161ea319689c192568c3167a92f1cf2e5c3a77c281c56824900f13c0125e992336c4a84dca9e0fe6f7c639faf4eb445eeedb18dcee260331f6dbec553da83ae8ea44092eb61eedda72d611a6a9e3e53e36589ca447de10ff1a56ddc522258a4b825961a33e85c520bc57eb42adb18b116013264d354d1f49913e96fe564b4a3b7c1d8d79fcdb92


Upon receiving the message and its signature, Bob can decrypt the signature **using the public key** and verify that it matches the message proving that Alice is indeed the sender.

In [54]:
long_to_bytes(FasterModularExponentiation(rsa_signature, ssh_public_key.e, ssh_public_key.n))

b'Hello Bob!'

# 5) Partial homomorphic encryption in "textbook" RSA

In this section, we show how unpadded RSA is a partially homomorphic encryption scheme.  

Let us consider two integers $x_1$ and $x_2$ on which we wish to perform some computation.  Instead of doing the computation locally, we want to perform it on some unsecure data processor (cloud for example...) without revealing to the remote server anything about our data other than the list of instructions to execute.

Let's define a small helper function to encrypt and decrypt data.  Notice that we can use the private key both for encryption and decryption. This is because for the use-case of homomorphic computation the remote server works directly on the encrypted ciphers and never needs to decrypt the data.

In [55]:
def encrypt_data(val, ssh_private_key):
    return ssh_private_key.encrypt(val, 'x')[0]

def decrypt_data(encrypted_val, ssh_private_key):
    return ssh_private_key.decrypt(encrypted_val)

Denoting by $\mathcal{E}$ the "textbook" RSA encryption function, the secret numbers $x_1$ and $x_2$ are encrypted into ciphers $c_1$ and $c_2$

\begin{align}
c_1 &= \mathcal{E}(x_1) \\
c_2 &= \mathcal{E}(x_2) 
\end{align}

In [56]:
x1 = 585
x2 = 117

In [57]:
c1 = encrypt_data(x1, ssh_private_key)
c2 = encrypt_data(x2, ssh_private_key)

In [58]:
print('x1 =', x1)
print('c1 =',hex(c1))

print('\nx2 =', x2)
print('c2 =', hex(c2))

x1 = 585
c1 = 0x57333c32bfcc36ab14dd3b90630b3a72eb31ea1ce8f5c7fdf39a99fd355ac09edceb3344308f51d9a264becc2d4fef63509b890a861bc9d73b3417d631f13fc4ac48d2ed9d86dd50e73ee682a3c3823bc6c487fcc5cb0cbea4bf86bdd6d7b30ead85a04c51657ed400addc326ff696d937a1c134db21a2041d1cf8440a3ac0a07f7a4badbaae8725780d3bd3732ce43aff6bd1bed8d33bb26bc8deae795ae52ff01d33b47ff0859ee353dae5e00d98d6aba20b008410811b08366a3dadd480df9fdb26e7702f9d9fe34eec8ad10a867d529bd49566e9b67ae56b031c9cef9db8270cc844cf24471cc29a1f46c1c3fdaa04635e90817e5d2496cfaccf168ad114

x2 = 117
c2 = 0x4fadda21e2f027c83deb4e9fe7857155c60e0236e2e19830f545635f830f2f975e1f1f896ee6df8b4e70ab4e3b4af466db36d8141d3e7472330d609488dceedb09766874452544b3b6222d03fcbb04929f3f8b8ce3d6310696d361610098e32b8445f804e1a42ae9a18a2c9ce0bebc56c02f8d65a285c6c4b34ebbc9b455e8bc9fcb81579e3ad462d6d6baa6daa563b65320513f177d2a3fe305faaf0377730b231d715932b95e84b0ffa80d6250ac0543ec3dbec60a329d76e3c0d3dbe55ba4e3401768a43da00cb222c0312d1a0522467a86c8fd81debff9be369104c970aff73404

The encrypted ciphers $c_2$ and $c_2$ can now be sent to an untrusted environment tasked with excecuting arithmetic instructions on the ciphers.

## 5.1) Multiplication

In [59]:
c1Mulc2 = c1 * c2

print('c1Mulc2 = ', hex(c1Mulc2))

c1Mulc2 =  0x1b24078083a6a4789d1e51718f5eee64cc29b008fa4a2075287852eede6f1487db8aaf24daa039abe20e9c16b1b594677c65bcb7a7b6e91e469ce4d0b442be8c92adf8b0055a6e6426082e45c7403fea59b439b5dfed5255d8ed1aa03b8b5e0e1200e8a8694eb919309c60e413be49bc32428a47daa9e9b3d31c7f1524628648da9af03387c7ae99107eae3dfa5d758b9c045ff8b3f90f44a1183c18e55137b360896242e128343f83d9a8a7b2a3f566461fdbfa5ffae62106b3ead0d58e5ce519e422d6f07c77bb925f0e24302353f99343b00628375feaea745e097da506e7905d2dfebaffdab3d1f1df99d9c03865211366c860d36f3d71260affbdf493a38518c54b569b9e4ebb169d7ae548d55b43674b7a6e33b538f98d0045b663025743099d94d5ae4912a68cf4c7e14d671ff3944b0007972b656bfdf0edf6cdfca20b81699c347c27f05d39faeaca1269cbcca06004e378689bb4b75335a4cd6a479999ebff2d9ecfb550f2c48a34f254e9a12a5c2ec19da9d8e9e3ac025fd65cc46bc94d59a53be989ce435539ad1480a8c0d22ed138aa81df7d94d34214215524142b81dc48ed20090d956665a9b94d2c7bcd3420a8f29c6ac33a9de5a266fdaec71b29c41fbd406f0a5f478828ba44a953fae74bdd2915c169596b9281bc97502720a2cf28bd3c2b76450f6cb76

Send the result of $c_1 \cdot c_2$ back to the data owner who is the only one able to decrypt

In [60]:
decrypt_data(c1Mulc2, ssh_private_key)

68445

which turns out to be exactly $x_1 \cdot x_2$ even though these numbers were never communicated to anyone

In [61]:
print(x1 * x2)

68445


**Why does this happen?**

For a more in-depth derivation, we refer to the accomapnying PDF document.  In a nutshell, it is easy to show that RSA encryption, denoted as $\mathcal{E}$ satisfies the multiplicative property: 

$$c_1 \cdot c_2 = \mathcal{E}(x_1) \cdot \mathcal{E}(x_2) = \mathcal{E}(x_1 \cdot x_2)$$

so that the decryption function $\mathcal{D}$ applied to the product $c_1\cdot c_2$ of the ciphers yields the product $x_1 \cdot x_2$ of the original two numbers.

$$\mathcal{D} (c_1 \cdot c_2) = \mathcal{D} (\mathcal{E}(x_1 \cdot x_2)) = x_1 \cdot x_2 $$

Notice that the untrusted environment never learned anything about $x_1, x_2$ or even about the result of $x_1 \cdot x_2$ since all it ever was exposed to were the encrypted ciphers $c_1, c_2$ and $c_1 \cdot c_2$.  Nevertheless, thanks to multiplicatice homomorphicity of RSA, it was able to perform a useful computation.

## 5.2) Division

Can we achieve the same thing with $x_1 \, /\, x_2$?  

First step is to determine the modular inverse of $x_2$ with respect to the public modulus $n$

In [62]:
x2_inverse = euclidAlg(x2, ssh_private_key.n)['fst_inv']

and to send its RSA encryption the untrusted server for a mutliplication with $c_1$

In [63]:
c2_inverse = encrypt_data(x2_inverse, ssh_private_key)

decrypt_data(c1 * c2_inverse, ssh_private_key)

5

In [64]:
print(x1 / x2)

5.0


Notice that for the sake of simplicity, we picked $x_1$ and $x_2$ so that their division would remain in the set of integers

## 5.3) Addition / subtraction

Unfortunately, these operations are not homomorphic under RSA and one needs to resort to different encryption schemes...

In [65]:
decrypt_data(c1 + c2, ssh_private_key)

23758146724011083768485928486553737473974099530048742568622783823486117923007924280248139398459642427759088128007224687390223099736026397438455997130905885190477227218619830218210711500445860041646749576575258193958165832022445040879233637591998742904400119411294846133975280016706710824114791989493808717686146183116808689674033413234126381542251594201182328142320060901938173134051143220784951396583168454712858174831083697730485767767231249708270691090089592232412646724737105418053445275440641541605215755954942120309856623932733010326566693869003044442258876977319503380238376931713747606844636871286187079245124

which is clearly very different from

In [66]:
x1 + x2

702

# 6) OAEP Padding 

See the accompanying PDF document for disucsssions about the security limitations of "textbook" RSA and how to solve them using a randomized padding scheme...